In [2]:
import numpy as np
from operator import xor
from IPython.display import clear_output

In [3]:
def permutate(bits, positions):
    permed = []
    for i in range(0, len(positions)):
        index = positions[i] - 1
        permed.append(int(bits[index]))
    return permed

In [4]:
def keyGeneration(raw_key):
    P10 = [3, 5, 2, 7, 4, 10, 1, 9, 8, 6]
    P8 = [6, 3, 7, 4, 8, 5, 10, 9]
    
    P10perm = permutate(raw_key, P10)
    P10LS1 = np.roll(P10perm[:5], -1)           #Left shift of 1
    P10LS2 = np.roll(P10perm[5:], -1)
    P10LS = list(P10LS1) + list(P10LS2)
    K1 = permutate(P10LS, P8)
    
    P10LS1 = np.roll(P10LS1, -2)              #Left shift of 2
    P10LS2 = np.roll(P10LS2, -2)
    P10LS = list(P10LS1) + list(P10LS2)
    K2 = permutate(P10LS, P8)
    keys = {
        "K1": K1,
        "K2": K2
    }
    
    return keys

In [5]:
def XOR(bits, SK):
    result = ''
    for i in range(0, len(bits)):
        result += str(int(bool(int(bits[i]) ^ int(SK[i]))))
    return list(result)

In [6]:
def f_k(bits, SK):
    EP = [4, 1, 2, 3, 2, 3, 4, 1]
    P4 = [2, 4, 3, 1]
    S0 = [[1, 0, 3, 2],
          [3, 2, 1, 0],
          [0, 2, 1, 3],
          [3, 1, 3, 2]]
  
    S1 = [[0, 1, 2, 3],
          [2, 0, 1, 3],
          [3, 0, 1, 0],
          [2, 1, 0, 3]]
    
    L = bits[:4]
    R = bits[4:]
    bits = permutate(R, EP)
    bits = XOR(bits, SK)
    
    rows = [int(bits[0] + bits[3], 2), int(bits[4] + bits[7], 2)]
    cols = [int(bits[1] + bits[2], 2), int(bits[5] + bits[6], 2)]
    LU_S0 = "{0:02b}".format(S0[rows[0]][cols[0]])
    LU_S1 = "{0:02b}".format(S1[rows[1]][cols[1]])
    bits = LU_S0 + LU_S1
    bits = permutate(bits, P4)
    return XOR(L, bits)

In [7]:
def encrypt(raw_key, plain_text):
    IP = [2, 6, 3, 1, 4, 8, 5, 7]
    IPinv = [4, 1, 3, 5, 7, 2, 8, 6]
    bits = permutate(plain_text, IP)
    keys = keyGeneration(raw_key)
    K1 = keys['K1']
    K2 = keys['K2']
    fk1 = f_k(bits, K1)
    bits = bits[4:] + fk1
    fk2 = f_k(bits, K2)
    encrypted = permutate(fk2 + fk1, IPinv)
    encrypted = "".join(str(i) for i in encrypted)
    return encrypted

In [13]:
encrypt('0010011111', '10100101')

'10010000'

In [9]:
def decrypt(raw_key, plain_text):
    IP = [2, 6, 3, 1, 4, 8, 5, 7]
    IPinv = [4, 1, 3, 5, 7, 2, 8, 6]
    bits = permutate(plain_text, IP)
    keys = keyGeneration(raw_key)
    K1 = keys['K1']
    K2 = keys['K2']
    fk1 = f_k(bits, K2)
    bits = bits[4:] + fk1
    fk2 = f_k(bits, K1)
    decrypted = permutate(fk2 + fk1, IPinv)
    decrypted = "".join(str(i) for i in decrypted)
    return decrypted

In [17]:
decrypt('1000101110', '11000010')

'00001100'

# Task 2 - Triple SDES

In [33]:
def TSDES_encrypt(raw_key1, raw_key2, plain_text):  #bascially following the 
    encrypt1 = encrypt(raw_key1, plain_text)        # formula given in task desc.
    decrypt1 = decrypt(raw_key2, encrypt1)
    encrypt2 = encrypt(raw_key1, decrypt1)
    return encrypt2

In [34]:
TSDES_encrypt('0000000000', '0000000000', '01010010')

'10000000'

In [35]:
def TSDES_decrypt(raw_key1, raw_key2, cipher_text):
    decrypt1 = decrypt(raw_key1, cipher_text)
    encrypt1 = encrypt(raw_key2, decrypt1)
    decrypt2 = decrypt(raw_key1, encrypt1)
    return decrypt2

In [39]:
TSDES_decrypt('0000000000', '0000000000', '11110000')

'00000000'

# Task 3 - Decryption with SDES

In [27]:
ctx1 = '01000111000000010100000011001101110010110000000101110100000000010110111001010111\
01010111011011100100011100000001010001111011101001001111100010000100011101101110\
01001100101011111001011101101110011011101011101001001111101011110000100101001010\
10001000010011111100110110010111010011110011001000000001010101110110111010010000\
01001111101011110100011110101111011101000111010000000001010011000000000101101110\
10111010100010000100011101101110010011001010111110010111000000011000100010010000'

In [28]:
def findpossiblekeys():
    possible_keys = []
    for i in range(0, 1023):
        key = "{0:010b}".format(i)
        possible_keys.append(str(key))
    return possible_keys

In [29]:
def splitcipher(cipher):
    ciphertext = cipher
    splitted = []
    while len(ciphertext)>0:
        string = ciphertext[0:8]
        splitted.append(string)
        ciphertext = ciphertext[8:]
    return splitted

In [30]:
def decryptSDES(cipher):
    possible_keys = findpossiblekeys()
    splitted = splitcipher(cipher)
    length = len(splitted)
    keys = {'Raw key   ': 'Plain text'}
    for key in possible_keys:
        text = ''
        valid = True
        for i in range(0, length):
            bits = decrypt(key, splitted[i])
            ASCII = int(bits, 2)
            if (ASCII>= ord('A') and ASCII <= ord('Z')) or (ASCII >= ord('a') and ASCII <= ord('z')):
                text += chr(ASCII)
            else:
                valid = False
                break
        if valid:
            keys[key] = text
    return keys

In [31]:
decryptSDES(ctx1)

{'Raw key   ': 'Plain text',
 '1111101010': 'simplifieddesisnotsecureenoughtoprovideyousufficientsecurity'}

# Task 3 - Decryption with Triple SDES

In [69]:
ctx2 = '00000001101001110011001011000110011001001010011111010111101001111001110001110100\
01110100100111000000000110100111000000011001100110100001110110100000000110011100\
11101111011111100010010010011100100111001001100110100001011111101010000010110011\
11011010101000011100011000100100101000010010001110100111011101001001110001000001\
10100001011111100000000101111110110101111101011110100111111011111010011110011100\
10011001110110100000000110011100111011110111111000100100101001111101101001000001'


In [105]:
def decryptTSDES(cipher):
    possible_keys1 = findpossiblekeys()
    possible_keys2 = findpossiblekeys()
    splitted = splitcipher(cipher)
    length = len(splitted)
    keys = []
    count = 0
    for key1 in possible_keys1:
        count += 1
        if count % 10 == 0:
            clear_output()
            print(round((count/1024)*100), '%')
        for key2 in possible_keys2:
            valid = True
            text = ''
            for i in range(0, length):
                bits = TSDES_decrypt(key1, key2, splitted[i])
                ASCII = int(bits, 2)
                if (ASCII>= ord('A') and ASCII <= ord('Z')) or (ASCII >= ord('a') and ASCII <= ord('z')):
                    text += chr(ASCII)
                else:
                    valid = False
                    break
            if valid:
                keys.append([key1, key2, text]) 
    return keys

In [106]:
decryptTSDES(ctx2)

100 %


[['1111101010',
  '0101011111',
  'simplifieddesisnotsecureenoughtoprovideyousufficientsecurity']]